### Add alarm configurations, per stream, into the `stream_confgs` table

In [ ]:
from decimal import Decimal

STANDARD_ALARM_CONFIG = {
    "default_channels": {
        "email": {"recipients": ["gustavofuhr87@gmail.com"]},
    },
    "rules": [
        {
            "enabled": True,
            "priority": 50,
            "expr": {
                "and": [
                    {">=": [ {"var": "object.person.max_confidence"}, Decimal("0.80") ]},
                    {">=": [ {"var": "object.dog.max_confidence"},    Decimal("0.75") ]},
                    {">=": [ {"var": "object.person.n_detections"}, Decimal(10) ]},
                    {">=": [ {"var": "object.dog.n_detections"},    Decimal(10) ]}
                ]
            },
            "cooldown_seconds": 30,
            "schedule": {
                "timezone": "Europe/Dublin",
                "include": [
                    {"days": ["mon","tue","wed","thu","fri","sat","sun"], "start": "17:00", "end": "07:00"}
                ]
            },
            "description": "Person walking a dog",
            "id": "person_walking_a_dog"
        },
        {
            "enabled": True,
            "priority": 40,
            "expr": {
                "and": [
                    {">=": [ { "var": "plate.162D11338.max_confidence" }, Decimal("0.70") ]},
                    {">=": [ { "var": "plate.162D11338.max_ocr_confidence" },   Decimal("0.70") ]}
                ]
            },
            "channels": {
                "whatsapp": {"numbers": ["+5551996039983"]}
            },
            "cooldown_seconds": 20,
            "description": "Notify when a known plate is detected",
            "id": "known_plate_alert"
        },
    ]
}

ANY_MOTION_ALARM_CONFIG = {
    "default_channels": {
        "whatsapp": {"numbers": ["+5551996039983"]},
        "email": {"recipients": ["gustavofuhr87@gmail.com"]},
    },
    "rules": [
        {
            "enabled": True,
            "priority": 10,
            "expr": {"var": "motion"},
            "cooldown_seconds": 60,
            "description": "Ping on any motion",
            "id": "any_motion_alert"
        }
    ]
}


STD_CAMERAS = ["B8A44FE6D078__booth_entrance", "B8A44FB3A1F9__front", "B8A44FE6C853__booth_exit", "B8A44FB981BB__panoramic_trains"]
ANY_MOTION_CAMERAS = ["B8A44FA0DC7A__axis-local"]


In [ ]:
import boto3
from datetime import datetime


dynamodb = boto3.resource("dynamodb", region_name="eu-west-1")
TABLE_NAME = "stream_configs"
table = dynamodb.Table(TABLE_NAME)

def put_ai_config(device_id, config_type, config_data):
    item = {
        "device_id": device_id,
        "config_type": config_type,
        "config": config_data,
        "updated_at": datetime.utcnow().isoformat()
    }

    table.put_item(Item=item)
    print(f"✅ Inserted config for device_id='{device_id}', config_type='{config_type}'")

for cam in STD_CAMERAS:
    put_ai_config(cam, "alarm_config_v2", STANDARD_ALARM_CONFIG)

for cam in ANY_MOTION_CAMERAS:
    put_ai_config(cam, "alarm_config_v2", ANY_MOTION_ALARM_CONFIG)
